In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('dates').getOrCreate()

In [3]:
df_stock = spark.read.csv(
    '../data/appl_stock.csv',
    inferSchema=True, header=True
)

In [4]:
df_stock = df_stock.withColumn('Date', df_stock['Date'].cast('timestamp'))
df_stock.printSchema()
df_stock.show(5)

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

+-------------------+----------+----------+------------------+------------------+---------+------------------+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|   

In [5]:
df_stock.head(1)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [6]:
from pyspark.sql.functions import dayofyear, dayofmonth, dayofweek, date_format, year, format_number

In [7]:
df_stock.select(df_stock['Date'], dayofmonth(df_stock['Date']), dayofyear(df_stock['Date']), dayofweek(df_stock['Date'])).show(10)

+-------------------+----------------+---------------+---------------+
|               Date|dayofmonth(Date)|dayofyear(Date)|dayofweek(Date)|
+-------------------+----------------+---------------+---------------+
|2010-01-04 00:00:00|               4|              4|              2|
|2010-01-05 00:00:00|               5|              5|              3|
|2010-01-06 00:00:00|               6|              6|              4|
|2010-01-07 00:00:00|               7|              7|              5|
|2010-01-08 00:00:00|               8|              8|              6|
|2010-01-11 00:00:00|              11|             11|              2|
|2010-01-12 00:00:00|              12|             12|              3|
|2010-01-13 00:00:00|              13|             13|              4|
|2010-01-14 00:00:00|              14|             14|              5|
|2010-01-15 00:00:00|              15|             15|              6|
+-------------------+----------------+---------------+---------------+
only s

In [8]:
high_low_by_year = df_stock.groupBy(year(df_stock['Date'])).mean('High', 'Low')
high_low_by_year = high_low_by_year.select(
    high_low_by_year['year(Date)'].alias('Year'),
    format_number('avg(High)', 2).alias('High avg'),
    format_number('avg(Low)', 2).alias('Low avg')
)
high_low_by_year = high_low_by_year.orderBy(high_low_by_year['Year'].desc())
high_low_by_year.show()

+----+--------+-------+
|Year|High avg|Low avg|
+----+--------+-------+
|2016|  105.43| 103.69|
|2015|  121.24| 118.86|
|2014|  297.56| 292.99|
|2013|  477.64| 468.25|
|2012|  581.83| 569.92|
|2011|  367.42| 360.30|
|2010|  262.37| 256.85|
+----+--------+-------+

